# Project milestone 4
# Detection of housing-health relationship


The aim of this project is to estimate the relation between housing quality and health status of a person. The project is an observational study based on a survey made by Mexican National Institute of Statistics and Geography. The health variables reported for each person will be used to define a single health variable which will be a score for each person's health status. We aim to use machine learning methods to do the classification and we will use regression models to predict this health score from the housing variables. Matching will be used to weed out the possible covariates. The motivation is to estimate the most important parameters of housing quality so that we can propose most cost-effective solutions that would increase the quality of health. The original paper is based on the analysis of influence of concrete floors on health quality, while here we would investigate some other parameters such as material used for building and whether there is a toilet or not in the household.

# Step 1: Import data

In [156]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

### Import data ###
#data_household = pd.read_csv('data_translated/household.csv')
data_house = pd.read_csv('data_translated/house.csv',low_memory=False)
data_person = pd.read_csv('data_translated/person.csv',low_memory=False)

In [157]:
#data_person_all = data_person.merge(data_house, left_on='House identifier', right_on='House identifier')
data_person_all = data_person.merge(data_house, left_on=['House_identifier'], right_on=['House_identifier'])

# Step 2: Exploratory Data Analysis

In [158]:
# our selected health variables
health_var = list(data_person_all.columns[35:57]) + [data_person_all.columns[26]]
data_person_all[health_var].describe()

,Difficulty_seeing,Wear_a_hearing_aid,Difficulty_hearing,Difficulty_hearing_without_noise,Difficulty_hearing_with_noise,Dificulty_to_walk,Use_a_walking_device,Walking_apparatus,Difficulty_walking_100_m,Difficulty_walking_500_m,...,Medication_for_nervousness,Intensity_of_nervousness,Frequency_of_depression,Antidepressant_medications,Intensity_of_depression,Frequency_of_pain,Pain_intensity,Fatigue_frequency,Tired_time,Limiting_physical_or_mental_activity
count,208140,208140,208140,208140,208140,208140,208140,208140,208140,208140,...,208140,208140,208140,208140,208140,208140,208140,208140,208140,208140
unique,6,3,6,6,6,6,3,9,6,6,...,4,5,7,4,5,6,5,6,5,2
top,1,2,1,1,1,1,2,,1,1,...,2,,5,2,,1,,1,,
freq,163575,197136,186322,190264,177606,175302,191991,201868,184590,177483,...,178203,117286,126698,180272,152071,132028,157153,129517,154635,205250


In [159]:
# get an idea of what kind of values are present in the daset
for i in range(len(data_person_all.columns)):
    print(data_person_all[data_person_all.columns[i]].isna().sum())
    print(data_person_all[data_person_all.columns[i]].value_counts())

0
1660272125    25
1360816210    20
102171202     20
1202467015    19
2460374003    19
              ..
3100686098     1
1060275044     1
2801063104     1
1801662205     1
2403427228     1
Name: House_identifier, Length: 56680, dtype: int64
0
1    205580
2      2185
3       322
4        46
5         7
Name: Household_identifier, dtype: int64
0
1     57518
2     51126
3     40348
4     28929
5     15849
6      7340
7      3481
8      1711
9       851
10      452
11      229
12      122
13       68
14       45
15       28
16       21
17       14
18        5
19        2
20        1
Name: Identifier_of_the_person, dtype: int64
0
17     4027
18     3996
8      3926
14     3915
12     3866
       ... 
101       3
110       2
103       2
106       1
120       1
Name: Age, Length: 107, dtype: int64
0
15    7629
12    7449
20    7421
10    7417
2     7417
1     7328
28    6971
4     6927
3     6922
8     6905
5     6855
17    6820
16    6808
6     6779
24    6758
7     6757
13    6738
18    672

2    180924
1     14895
      12321
Name: Used_bedroom_kitchen, dtype: int64
0
2     87191
3     55402
1     45160
4     15228
5      3685
6      1105
7       234
8        69
11       29
9        29
10        5
15        3
Name: Bedroom_rooms, dtype: int64
0
4     56059
3     46091
5     38621
2     23915
6     19150
1      9177
7      8339
8      3860
9      1440
10      839
11      312
12      214
13       54
14       29
15       20
16       16
19        3
17        1
Name: Number_of_rooms, dtype: int64
0
1    150216
2     44333
6      7726
5      3054
4      2404
3       407
Name: Water_availability, dtype: int64
0
1    142057
2     28974
      13591
3     10154
4      7312
5      6052
Name: Water_supply, dtype: int64
0
1    201703
2      6437
Name: Has_a_toilet, dtype: int64
0
2    193536
1      8167
       6437
Name: Shared_use_of_the_toilet, dtype: int64
0
1    141473
2     50995
3      9235
       6437
Name: Sanitary_water_connection, dtype: int64
0
1    108498
0     57411
2    

      192451
2      13854
1        999
3        426
4        275
5         54
7         41
6         24
14        16
Name: Secon_owner_identifier, dtype: int64
0
     192451
1     15647
2        30
3        12
Name: Second_owner's_home, dtype: int64
0
1    102782
      70164
3     30110
2      3739
9      1345
Name: Property_deeds, dtype: int64
0
2    133096
1     75044
Name: Computer_availability, dtype: int64
0
2    138120
1     70020
Name: Fixed_telephone_line_availability, dtype: int64
0
1    187816
2     20324
Name: Cell_phone_availability, dtype: int64
0
2    110840
1     97300
Name: Internet_availability, dtype: int64
0
1    110301
2     97839
Name: Pay_TV_service_availability, dtype: int64
0
2    111940
1     96200
Name: Availability_of_own_car, dtype: int64
0
1    203232
2      3884
3       810
4       180
5        34
Name: Total_households_in_the_dwelling, dtype: int64
0
10010000     4498
220140000    3180
230050000    2998
310500000    2814
20040000     2622
             ...

In [160]:
# replacing " " and '9' values with proper Nan
data_person_all = data_person_all.replace(' ', np.nan)
data_person_all = data_person_all.replace('&', np.nan)
data_person_all = data_person_all.replace('9', np.nan)

In [161]:
# drop columns with more than 30% of NaN values
nulli = []
exc = []
thr = int(len(data_person_all)*0.3)
for i in range(len(list(data_person_all.columns))):
    nulli.append(data_person_all[list(data_person_all.columns)[i]].isnull().values.sum())
    if (nulli[i] > thr):
        exc.append(i)
data_person_all = data_person_all.drop(columns = [list(data_person_all.columns)[i] for i in exc], axis=1)
# drop columns considere not useful
data_person_all = data_person_all.drop(columns = ["Birthday", "Birth_month", "Basic_geostatistical_area"], axis=1)

In [162]:
# update health variable
health_var = [i for i in health_var if i in list(data_person_all.columns)]
len(health_var)

17

### Filter nan values

In [163]:
# fill nan values with median should be more robust than mean
data_person_all.fillna(data_person_all.median(),inplace = True)

### Choose apropriate data type for variables and categorize

In [164]:
# converting our data to integer type
for temp in data_person_all.columns:
    data_person_all[temp]=data_person_all[temp].astype(int)

House_identifier
Household_identifier
Identifier_of_the_person
Age
Sex
Relationship
School_attendance
Grade_level_of_instruction
Level_of_instruction
Home
Literacy
Marital_status
Worked_last_week
Wear_glasses
Difficulty_seeing
Wear_a_hearing_aid
Difficulty_hearing
Difficulty_hearing_without_noise
Difficulty_hearing_with_noise
Dificulty_to_walk
Use_a_walking_device
Difficulty_walking_100_m
Difficulty_walking_500_m
Difficulty_climbing_12_steps_
Difficulty_remembering
Frequency_of_nervousness
Medication_for_nervousness
Frequency_of_depression
Antidepressant_medications
Frequency_of_pain
Fatigue_frequency
Housing_type
Wall_material
Roofing_material
Flooring_material
Roof_insulation
Insulation_in_walls
Window_insulation
Other_type_of_insulation
Age_of_the_home
It_has_kitchen
Used_bedroom_kitchen
Bedroom_rooms
Number_of_rooms
Water_availability
Water_supply
Has_a_toilet
Shared_use_of_the_toilet
Sanitary_water_connection
Toilet_toilet_shower
Toilet_toilet
Toilet_shower
Drainage_destination
El

# Step 3: Regression

### Fill response variables

In [146]:
# To run for all health_var or for health score
response=health_var[0]

### Fill model with all variables

In [147]:
# Do we filter here only important variables for example based on person correlation coef. for each response?

model='~Age'
start=data_person_all.columns.get_loc('Sex')
end=data_person_all.columns.get_loc('Worked_last_week')

for index,item in enumerate(data_person_all.columns):
    # Fill with person (control) variables
    if index >=start and index <=end:
        model=model+'+'+item
        #model=model+'+'+'C('+item+')'

    # To add also House variables
    elif index>end:
        break

In [148]:
formula=""
formula=response+model

In [149]:
# Fits the model (find the optimal coefficients, adding a random seed ensures consistency)
np.random.seed(1950)
mod= smf.ols(formula=response+model, data=data_person_all,missing='raise')
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      Difficulty_seeing   R-squared:                       0.140
Model:                            OLS   Adj. R-squared:                  0.139
Method:                 Least Squares   F-statistic:                     527.4
Date:                Wed, 16 Dec 2020   Prob (F-statistic):               0.00
Time:                        15:45:24   Log-Likelihood:            -1.2580e+05
No. Observations:              208140   AIC:                         2.517e+05
Df Residuals:                  208075   BIC:                         2.524e+05
Df Model:                          64                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                           0.9814      0.006    165.973      0.000       0.970       0.993
School_attendance[T.1]           1.327e+09   8.63e+09      0.154      0.878   -1.56e+10    1.82e+10
School_attendance[T.2]           1.327e+09   8.63e+09      0.154      0.878   -1.56e+10    1.82e+10
Grade_level_of_instruction[T.0] -6.689e+08   4.35e+09     -0.154      0.878   -9.19e+09    7.86e+09
Grade_level_of_instruction[T.1] -1.327e+09   8.63e+09     -0.154      0.878   -1.82e+10    1.56e+10
Grade_level_of_instruction[T.2] -1.327e+09   8.63e+09     -0.154      0.878   -1.82e+10    1.56e+10
Grade_level_of_instruction[T.3] -1.327e+09   8.63e+09     -0.154      0.878   -1.82e+10    1.56e+10
Grade_level_of_instruction[T.4] -1.327e+09   8.63e+09     -0.154      0.878   -1.82e+10    1.56e+10
Grade_level_of_instruction[T.5] -1.327e+09   8.63e+09     -0.154      0.878   -1.82e+10    1.56e+10
Grade_level_of_instruction[T.6] -1.327e+09   8.63e+09     -0.154      0.878   -1.82e+10    1.56e+10
Level_of_instruction[T.0]       -6.578e+08   4.28e+09     -0.154      0.878   -9.04e+09    7.73e+09
Level_of_instruction[T.1]           0.1526      0.016      9.704      0.000       0.122       0.183
Level_of_instruction[T.10]          0.0301      0.014      2.175      0.030       0.003       0.057
Level_of_instruction[T.11]         -0.0523      0.017     -3.035      0.002      -0.086      -0.019
Level_of_instruction[T.12]         -0.1406      0.028     -4.956      0.000      -0.196      -0.085
Level_of_instruction[T.2]           0.1733      0.014     12.343      0.000       0.146       0.201
Level_of_instruction[T.3]           0.0937      0.014      6.690      0.000       0.066       0.121
Level_of_instruction[T.4]           0.0713      0.014      5.064      0.000       0.044       0.099
Level_of_instruction[T.5]           0.0107      0.022      0.490      0.624      -0.032       0.054
Level_of_instruction[T.6]          -0.0479      0.022     -2.131      0.033      -0.092      -0.004
Level_of_instruction[T.7]           0.0315      0.016      2.005      0.045       0.001       0.062
Level_of_instruction[T.8]           0.0166      0.017      0.971      0.332      -0.017       0.050
Home[T.1]                          -0.0453      0.008     -5.981      0.000      -0.060      -0.030
Home[T.10]                          0.0267      0.008      3.509      0.000       0.012       0.042
Home[T.11]                          0.0082      0.007      1.123      0.262      -0.006       0.022
Home[T.12]                         -0.0125      0.008     -1.660      0.097      -0.027       0.002
Home[T.13]                          0.0287      0.008      3.735      0.000       0.014       0.044
Home[T.14]                         -0.0240      0.008     -3.191      0.001      -0